In [1]:
import pandas as pd
import pyomo.environ as pyo
from collections import defaultdict
# conda install -c conda-forge coincbc
# conda install -c conda-forge ipopt

In [2]:
counties = pd.read_csv('data/oh_county_list.csv')
counties['county_id'] = pd.Series(range(1,89))
df_pop = pd.read_csv('data/oh_county_pop_2021.csv')
df = pd.merge(counties, df_pop, on='county')

In [3]:
with open('data/oh_adj_loc.dat', 'r') as f:
    read_adj = f.readlines()
sublist = list(map(lambda x: x.strip('\n').split(','), read_adj))
adj_loc = [[int(x) for x in sublst] for sublst in sublist]
df = pd.concat([df, pd.Series(adj_loc, name='adj_id')], axis=1)

In [4]:
adj_matrix = defaultdict(list)
for i in range(1,89):
    adj_matrix[i] = adj_loc[i-1]
adj_matrix 

defaultdict(list,
            {1: [8, 36, 66, 73],
             2: [6, 32, 33, 69, 81],
             3: [38, 39, 42, 47, 52, 70, 85],
             4: [28, 43, 78],
             5: [37, 53, 58, 64, 82, 84],
             6: [2, 19, 33, 46, 54, 75, 81],
             7: [30, 34, 41, 56, 61],
             8: [1, 13, 14, 36],
             9: [31, 57, 68, 83],
             10: [15, 34, 41, 76, 79],
             11: [12, 46, 49, 55, 75, 80],
             12: [11, 29, 49, 55, 57],
             13: [8, 14, 31, 83],
             14: [8, 13, 24, 29, 36, 83],
             15: [10, 41, 50, 76],
             16: [30, 38, 42, 45, 60, 79],
             17: [39, 51, 59, 70, 74, 88],
             18: [28, 43, 47, 52, 77],
             19: [6, 54, 55, 57, 68, 75],
             20: [35, 63, 69, 86],
             21: [25, 42, 45, 51, 59, 80],
             22: [39, 47, 72],
             23: [25, 37, 45, 64, 65],
             24: [14, 29, 36, 49, 65, 71],
             25: [21, 23, 45, 49, 65, 80],
           

In [5]:
def county_list_to_names(id_list, counties):
    return counties[counties['county_id'].isin(id_list)]['county'].tolist()
df['adj_names'] = df['adj_id'].apply(lambda x: county_list_to_names(x, counties))

In [6]:
df

,county,county_id,population,adj_id,adj_names
0,Adams County,1,27564,"[8, 36, 66, 73]","[Brown County, Highland County, Pike County, S..."
1,Allen County,2,102462,"[6, 32, 33, 69, 81]","[Auglaize County, Hancock County, Hardin Count..."
2,Ashland County,3,52658,"[38, 39, 42, 47, 52, 70, 85]","[Holmes County, Huron County, Knox County, Lor..."
3,Ashtabula County,4,97869,"[28, 43, 78]","[Geauga County, Lake County, Trumbull County]"
4,Athens County,5,62933,"[37, 53, 58, 64, 82, 84]","[Hocking County, Meigs County, Morgan County, ..."
...,...,...,...,...,...
83,Washington County,84,59942,"[5, 56, 58, 61]","[Athens County, Monroe County, Morgan County, ..."
84,Wayne County,85,116858,"[3, 38, 52, 76, 77]","[Ashland County, Holmes County, Medina County,..."
85,Williams County,86,37026,"[20, 26, 35]","[Defiance County, Fulton County, Henry County]"
86,Wood County,87,131930,"[32, 35, 48, 62, 72, 74]","[Hancock County, Henry County, Lucas County, O..."


In [7]:
model = pyo.ConcreteModel()

# declare decision variables
model.x = pyo.Var(domain=pyo.NonNegativeReals)

# declare objective
model.profit = pyo.Objective(
    expr = 40*model.x,
    sense = pyo.maximize)

# declare constraints
model.demand = pyo.Constraint(expr = model.x <= 40)
model.laborA = pyo.Constraint(expr = model.x <= 80)
model.laborB = pyo.Constraint(expr = 2*model.x <= 100)

# solve
pyo.SolverFactory('cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1600.0
  Upper bound: 1600.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 2
  Number of nonzeros: 0
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created s

In [25]:
def param_adj(m, i, j):
    return int(m.j in adj_matrix[m.i])

def param_pop(m, i):
    return df['population'][i-1]

In [26]:
model = pyo.AbstractModel()

# value of n (number of counties)
model.n = pyo.Param(initialize=88)

# range of i and j (iterating over counties)
model.I = pyo.RangeSet(1, model.n)
model.J = pyo.RangeSet(1, model.n)

# TOSET limit on number of pirincipal places of buisnesses opened (init to 5)
model.k = pyo.Var(within=pyo.NonNegativeIntegers, initialize=5)

model.p = pyo.Param(model.I, initialize=param_pop) # population of county i
model.a = pyo.Var(model.I, model.J, domain=pyo.Binary, initialize=param_adj) # 1 if county i and j are adjacent
model.x = pyo.Var(model.J, domain=pyo.Binary) # 1 if principal place of business is opened in county j
model.y = pyo.Var(model.I, domain=pyo.Binary) # 1 if county i is covered

In [27]:
def obj_sum(m):
    return pyo.summation(m.p, m.y)

def con_a(m, i):
    return sum(m.a[i,j] * m.x[j] for j in m.J) >= m.y[i]

def con_x(m, j):
    return m.x[j] <= m.k

# def param_pop(m, i, j):
#     return int(m.j in adj_matrix[m.i])

In [28]:
model.obj = pyo.Objective(
    rule=obj_sum,
    sense = pyo.maximize)

model.a_constraint = pyo.Constraint(model.I, rule=con_a)
model.x_constraint = pyo.Constraint(model.J, rule=con_x)

In [29]:
data = {None: {
    'I': {None: [1,2,3]},
    'p': {None: 100},
    'q': {1: 10, 2:20, 3:30},
    'r': {(1,1): 110, (1,2): 120, (2,3): 230},
}}
instance = model.create_instance()

ERROR: Rule failed when initializing variable for Var a with index None:
    AttributeError: 'AbstractModel' object has no attribute 'j'
ERROR: Constructing component 'a' from data=None failed:
        AttributeError: 'AbstractModel' object has no attribute 'j'


AttributeError: 'AbstractModel' object has no attribute 'j'